## Check GPU
Mixed precision training need GPu with compute capability of 7.0+.

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-25824f3f-d8e8-604b-a4ea-62f14d22994d)


In [2]:
## Mix precision training have error with tensorflow 2.5+, so we use tensorflow 2.4.1
!pip install tensorflow==2.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.3 MB 13 kB/s 
     |████████████████████████████████| 132 kB 63.9 MB/s 
     |████████████████████████████████| 3.8 MB 51.6 MB/s 
     |████████████████████████████████| 462 kB 68.0 MB/s 
     |████████████████████████████████| 14.8 MB 51.4 MB/s 
     |████████████████████████████████| 2.9 MB 45.0 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68720 sha256=de56f1558f69511ddd6339c97ba8cdc96b1f90a08f49954b9100fb8321cb52f5
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: numpy
    Found existing installation: n

In [2]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [1]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

/Users/chientm/miniconda3/envs/sandbox/lib/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


## Use TensorFlow Datasets to Download Data

In [4]:
import tensorflow_datasets as tfds

In [5]:
datasets_list = tfds.list_builders() # get all available datasets in TFDS

True


In [6]:
(train_data, test_data), ds_info = tfds.load(name="food101", # target dataset to get from TFDS
                                             split=["train", "validation"], # what splits of data should we get? note: not all datasets have train, valid, test
                                             shuffle_files=True, # shuffle files on download?
                                             as_supervised=True, # download data in tuple format (sample, label), e.g. (image, label)
                                             with_info=True) # include dataset metadata? if so, tfds.load() returns tuple (data, ds_info)

In [7]:
ds_info.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=101),
})

In [8]:
class_names = ds_info.features["label"].names
class_names[:10]

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito']

## Create preprocessing functions for our data

In [14]:
def preprocess_img(image, label, img_shape=224):
  """
  Converts image datatype from 'uint8' -> 'float32' and reshapes image to
  [img_shape, img_shape, color_channels]
  """
  image = tf.image.resize(image, [img_shape, img_shape]) # reshape to img_shape
  return tf.cast(image, tf.float32), label # return (float32_image, label) tuple

## Batch & prepare datasets

We're going to be using:

* [`map()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) - maps a predefined function to a target dataset (e.g. `preprocess_img()` to our image tensors)
* [`shuffle()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle) - randomly shuffles the elements of a target dataset up `buffer_size` (ideally, the `buffer_size` is equal to the size of the dataset, however, this may have implications on memory)
* [`batch()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch) - turns elements of a target dataset into batches (size defined by parameter `batch_size`)
* [`prefetch()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#prefetch) - prepares subsequent batches of data whilst other batches of data are being computed on (improves data loading speed but costs memory)
* Extra: [`cache()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#cache) - caches elements in a target dataset, saving loading time (will only if your dataset is small enough to fit in memory, standard Colab instances only have 12GB of memory) 

Things to note:
- `shuffle()` keeps a buffer of the number you pass it images shuffled, ideally this number would be all of the samples in your training set, however, if your training set is large, this buffer might not fit in memory (a fairly large number like 1000 or 10000 is usually suffice for shuffling)
- For methods with the `num_parallel_calls` parameter available (such as `map()`), setting it to`num_parallel_calls=tf.data.AUTOTUNE` will parallelize preprocessing and significantly improve speed
- Can't use `cache()` unless your dataset can fit in memory

Woah, the above is alot. But once we've coded below, it'll start to make sense.

We're going to through things in the following order:

```
Original dataset (e.g. train_data) -> map() -> shuffle() -> batch() -> prefetch() -> PrefetchDataset
```

In [17]:
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)
train_data, test_data

## Create modelling callbacks

In [19]:
# TensorBoard callback
from helper_functions import create_tensorboard_callback

# ModelCheckpoint callback
checkpoint_path = "model_checkpoints/feat_extractor/cp.ckpt" # saving weights requires ".ckpt" extension
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      montior="val_acc", # save the model weights with best validation accuracy
                                                      save_best_only=True, # only save the best weights
                                                      save_weights_only=True, # only save model weights (not whole model)
                                                      verbose=0) # don't print out whether or not model is being saved 

## Setup mixed precision training

In [20]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy="mixed_float16")

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


In [21]:
mixed_precision.global_policy() # should output "mixed_float16"

<Policy "mixed_float16">

## Build feature extraction model

In [22]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

inputs = layers.Input(shape=input_shape, name="input_layer")
# Note: EfficientNetBX models have rescaling built-in but if your model didn't you could have a layer like below
# x = preprocessing.Rescaling(1./255)(x)
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x = layers.Dense(len(class_names))(x)
# Separate activation of output layer so we can output float32 activations
outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x) 
model = tf.keras.Model(inputs, outputs)

model.compile(loss="sparse_categorical_crossentropy", # Use sparse_categorical_crossentropy when labels are *not* one-hot
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [23]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
pooling_layer (GlobalAverage (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 101)               129381    
_________________________________________________________________
softmax_float32 (Activation) (None, 101)               0         
Total params: 4,178,952
Trainable params: 129,381
Non-trainable params: 4,049,571
_________________________________________________________________


In [24]:
for layer in model.layers:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy) # Check the dtype policy of layers

input_layer True float32 <Policy "float32">
efficientnetb0 False float32 <Policy "mixed_float16">
pooling_layer True float32 <Policy "mixed_float16">
dense True float32 <Policy "mixed_float16">
softmax_float32 True float32 <Policy "float32">


## Fit the feature extraction model

In [26]:
# Fit the feature extraction model for 3 epochs with tensorboard and model checkpoint callbacks
history_feat_extractor = model.fit(train_data, epochs=3, validation_data=test_data, validation_steps=int(0.15*len(test_data)),
                                   callbacks=[create_tensorboard_callback(dir_name='tensorboard_logs', experiment_name='feat_extract'), model_checkpoint])

Saving TensorBoard log files to: tensorboard_logs/feat_extract/20220624-075601
Epoch 1/3
2368/2368 [==============================] - 211s 77ms/step - loss: 2.3259 - accuracy: 0.4647 - val_loss: 1.2352 - val_accuracy: 0.6758
Epoch 2/3
2368/2368 [==============================] - 175s 73ms/step - loss: 1.3110 - accuracy: 0.6621 - val_loss: 1.1277 - val_accuracy: 0.6992
Epoch 3/3
2368/2368 [==============================] - 203s 85ms/step - loss: 1.1501 - accuracy: 0.6992 - val_loss: 1.0809 - val_accuracy: 0.7129


In [27]:
# Evaluate model (unsaved version) on whole test dataset
feat_extract_res = model.evaluate(test_data)
feat_extract_res

790/790 [==============================] - 80s 101ms/step - loss: 1.0881 - accuracy: 0.7078


[1.0880811214447021, 0.7077623605728149]

## Save the whole model to file

In [30]:
# Save model locally (if you're using Google Colab, your saved model will Colab instance terminates)
model.save('feat_extractor_trained')

INFO:tensorflow:Assets written to: feat_extractor_trained/assets


INFO:tensorflow:Assets written to: feat_extractor_trained/assets


In [ ]:
# Load model previously saved above
loaded_model = tf.keras.models.load_model('feat_extractor_trained')

In [33]:
# Check the layers in the base model and see what dtype policy they're using
# Check the dtype_policy attributes of layers in our model
for layer in loaded_model.layers:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy) # Check the dtype policy of layers

input_layer True float32 <Policy "float32">
efficientnetb0 True float32 <Policy "mixed_float16">
pooling_layer True float32 <Policy "mixed_float16">
dense True float32 <Policy "mixed_float16">
softmax_float32 True float32 <Policy "float32">


In [36]:
# Check loaded model performance (this should be the same as results_feature_extract_model)
loaded_feat_extract_res = loaded_model.evaluate(test_data)
loaded_feat_extract_res

790/790 [==============================] - 68s 83ms/step - loss: 1.0926 - accuracy: 0.7041


[1.0925607681274414, 0.7040792107582092]

In [37]:
import numpy as np
np.isclose(feat_extract_res, loaded_feat_extract_res)

array([ True,  True])

## Preparing our model's layers for fine-tuning

In [28]:
# Set all of the layers .trainable variable in the loaded model to True (so they're unfrozen)
model.trainable = True

In [29]:
for layer in model.layers:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy) # Check the dtype policy of layers

input_layer True float32 <Policy "float32">
efficientnetb0 True float32 <Policy "mixed_float16">
pooling_layer True float32 <Policy "mixed_float16">
dense True float32 <Policy "mixed_float16">
softmax_float32 True float32 <Policy "float32">


In [30]:
# EarlyStopping callback
early_stopping_callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=3)


# ModelCheckpoint callback
checkpoint_path = "model_checkpoints/finetuned_model/cp.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, montior="val_loss", mode='min',
                                                      save_best_only=True, save_weights_only=True, verbose=0)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=2,
                                                 min_lr=1e-7, verbose=1)

In [31]:
# Use the Adam optimizer with a 10x lower than default learning rate
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0001), metrics=['accuracy'])

In [33]:
# Start to fine-tune (all layers)
# Use 100 epochs as the default, Validate on 15% of the test_data
finetune_history = model.fit(train_data, epochs=100, initial_epoch=history_feat_extractor.epoch[-1],
                             validation_data=test_data, validation_steps=int(0.15*len(test_data)),
                             callbacks=[create_tensorboard_callback(dir_name='tensorboard_logs', experiment_name='finetune_model'),
                                        model_checkpoint, early_stopping_callbacks, reduce_lr])


Saving TensorBoard log files to: tensorboard_logs/finetune_model/20220624-080725
Epoch 3/100
2368/2368 [==============================] - 356s 142ms/step - loss: 0.9829 - accuracy: 0.7368 - val_loss: 0.8057 - val_accuracy: 0.7728
Epoch 4/100
2368/2368 [==============================] - 291s 122ms/step - loss: 0.5776 - accuracy: 0.8411 - val_loss: 0.7817 - val_accuracy: 0.7873
Epoch 5/100
2368/2368 [==============================] - 291s 122ms/step - loss: 0.3261 - accuracy: 0.9080 - val_loss: 0.8727 - val_accuracy: 0.7773
Epoch 6/100
2368/2368 [==============================] - 291s 122ms/step - loss: 0.1648 - accuracy: 0.9520 - val_loss: 0.9651 - val_accuracy: 0.7773

Epoch 00006: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.
Epoch 7/100
2368/2368 [==============================] - 290s 122ms/step - loss: 0.0508 - accuracy: 0.9885 - val_loss: 1.0222 - val_accuracy: 0.8085


In [34]:
model.save('finetune_model')

INFO:tensorflow:Assets written to: finetune_model/assets


INFO:tensorflow:Assets written to: finetune_model/assets


In [35]:
finetune_res = model.evaluate(test_data)
finetune_res

790/790 [==============================] - 50s 64ms/step - loss: 1.0164 - accuracy: 0.8015


[1.0164353847503662, 0.8015049695968628]

## View training results on TensorBoard

In [36]:
!tensorboard dev upload --logdir ./tensorboard_logs \
  --name "Fine-tuning EfficientNetB0 on all Food101 Data" \
  --description "Training results for fine-tuning EfficientNetB0 on Food101 Data with learning rate 0.0001" \
  --one_shot

2022-06-24 08:44:20.659551: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./tensorboard_logs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A

## More evaluate trained model

Some ideas you might want to go through:
1. Find the precision, recall and f1 scores for each class (all 101).
2. Build a confusion matrix for each of the classes.
3. Find your model's *most wrong* predictions (those with the highest prediction probability but the wrong prediction).

See the evaluation section at the end of [Transfer Learning Part 3: Scaling Up for more](https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/06_transfer_learning_in_tensorflow_part_3_scaling_up.ipynb).

In [ ]:
# YOUR_CODE_HERE